# 1D Variability hypothesis testing for lupus data

In [1]:
import scanpy as sc
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
from pybedtools import BedTool
import pickle as pkl
%matplotlib inline

In [10]:
pd.set_option('display.max_columns', None)

In [ ]:
import sys
sys.path.append('/home/ssm-user/Github/scrna-parameter-estimation/dist/memento-0.0.6-py3.8.egg')
sys.path.append('/home/ssm-user/Github/misc-seq/miscseq/')
import encode
import memento

In [9]:
data_path = '/data_volume/memento/lupus/'

### Read the processed RNA data

Focus on the club and bc/club cells and type I interferons for now.

Encode the timestamps to integers.

In [4]:
adata = sc.read(data_path + 'Lupus_study_adjusted_2020_new2_5000genes_batch_fixed_noplatelet.h5ad')

/home/ssm-user/anaconda3/envs/single_cell/lib/python3.8/site-packages/anndata/compat/__init__.py:179: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(
/home/ssm-user/anaconda3/envs/single_cell/lib/python3.8/site-packages/anndata/compat/__init__.py:179: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where adjacency matrices should go now.
  warn(


In [13]:
adata_count = sc.read(data_path + 'Lupus_study_2020.h5ad')

### Select cells


In [15]:
adata.obs.batch.value_counts()

0    1244585
1      19091
Name: batch, dtype: int64

In [17]:
L3_cMs = adata.obs.query('Processing_Cohort == "L3" & cg_cov == "cM"')

In [36]:
healthy_inds = L3_cMs.query('SLE_status == "Healthy"').ind_cov.value_counts().head(10).index.tolist()
sle_inds = L3_cMs.query('SLE_status == "SLE"').ind_cov.value_counts().head(10).index.tolist()

In [37]:
barcodes = L3_cMs.query('ind_cov in @healthy_inds | ind_cov in @sle_inds').index.tolist()

In [44]:
count_bcs = adata_count.obs.index.tolist()
overlap = list(set(count_bcs) & set(barcodes))

In [45]:
filtered_counts = adata_count[overlap, :]

/home/ssm-user/anaconda3/envs/single_cell/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


In [50]:
filtered_counts.obs = L3_cMs.loc[overlap, :]

In [52]:
filtered_counts.write(data_path + 'cM_sampled.h5ad')